# Plate Solve a Synthetic Image (Using FITS Header RA/Dec, plus Alt/Az)

This notebook demonstrates how to plate solve a synthetic image by:
1. **Reading RA/Dec from the FITS header** to guide the star catalog search.
2. **Detecting stars** using segmentation-based methods (Photutils).
3. **Querying the star catalog** near the header coordinates.
4. **Astroalign** to match the detected stars to the catalog's tangent-plane projection.
5. **Directly use the header RA/Dec** for the tangent-plane center (i.e., no median-based shift).
6. **Computing** the telescope’s *Alt/Az* from the same RA/Dec, the observation time, and Earth location for debugging.
7. **Overlay a final debug plot** showing the entire plate-solved starfield on top of the image.

We do **not** save a final plate-solved FITS or reproject the data. We only compute it for debugging.


In [1]:
# If needed, install dependencies:
# %pip install astropy photutils astroalign skyfield

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table
from astropy.stats import SigmaClip
from photutils.background import Background2D, MedianBackground
from photutils.segmentation import detect_sources, deblend_sources, SourceCatalog
from photutils.aperture import CircularAperture
import astroalign
import os
from astropy.coordinates import SkyCoord
import astropy.units as u
from skyfield.api import load, wgs84, Star
from datetime import datetime

# Local function to load the star catalog
from synthetic_image import get_star_catalog

############################################
# Configuration
############################################

fits_file_pattern = 'output/synthetic_image_*.fits'  # Glob for synthetic images
# fits_file_pattern = 'Light_1509.fits'  # Glob for synthetic images
star_catalog_fits = 'celestial_equator_band.fits'    # Local star catalog file

# Catalog search radius in degrees
search_radius_deg = 3.0
max_mag = 16.0          # We'll gather stars up to this magnitude.

# Photutils detection parameters
threshold_sigma = 2.0     # Lower to catch fainter stars
npixels = 8               # Min connected pixels
deblend_nthresh = 32      # Deblend parameters
deblend_contrast = 0.1

# Aperture radius for plotting
AP_RAD = 10.0

# Observatory location
obs_lat_deg = 34.05    # e.g. Los Angeles area
obs_lon_deg = -118.25
obs_elev_m = 100.0

############################################
# 1) Load the synthetic image
############################################
import glob
matched_files = glob.glob(fits_file_pattern)
if len(matched_files) == 0:
    raise FileNotFoundError('No synthetic FITS file found. Check your path or generate a new one.')
matched_files.sort(key=os.path.getmtime)
fits_path = matched_files[-1]
print(f'Using synthetic FITS: {fits_path}')

hdul = fits.open(fits_path)
data = hdul[0].data.astype(float)
header = hdul[0].header
hdul.close()

# Attempt to read RA, Dec from the header.
# We assume either RA/DEC or CRVAL1/CRVAL2 might be present, in degrees.
header_ra = header.get('RA', header.get('CRVAL1', 0.0))
header_dec = header.get('DEC', header.get('CRVAL2', 0.0))

print(f"Header-based approximate center: RA={header_ra:.4f}, Dec={header_dec:.4f}")

# Attempt to read the observation time from DATE-OBS
header_dateobs = header.get('DATE-OBS', None)
if header_dateobs:
    try:
        obs_time = datetime.fromisoformat(header_dateobs)
    except ValueError:
        # fallback if not parseable
        obs_time = datetime(2025,1,10,9,0,0)
else:
    obs_time = datetime(2025,1,10,9,0,0)

print(f"Observation time (DATE-OBS): {obs_time}")

plt.figure(figsize=(8,6))
plt.imshow(data, origin='upper', cmap='gray',
           vmin=np.percentile(data, 0.5), vmax=np.percentile(data, 99.5))
plt.title('Loaded Synthetic Image')
plt.colorbar(label='Counts')
plt.show()

## 2) Detect Stars with Segmentation


In [ ]:
# Background subtraction
sigma_clip = SigmaClip(sigma=3.0)
bkg_estimator = MedianBackground()
bkg = Background2D(data, (50, 50), filter_size=(3, 3), sigma_clip=sigma_clip,
                  bkg_estimator=bkg_estimator)

data_sub = data - bkg.background
threshold_value = threshold_sigma * bkg.background_rms

# Segment the image
segm = detect_sources(data_sub, threshold_value, npixels=npixels)
if segm is None:
    raise RuntimeError('No sources detected. Try lowering threshold_sigma or adjusting npixels.')

# Deblend sources
segm_deblend = deblend_sources(data_sub, segm, npixels=npixels,
                               nlevels=deblend_nthresh,
                               contrast=deblend_contrast)

# Build source catalog
cat = SourceCatalog(data_sub, segm_deblend)
tbl = cat.to_table()
print(f'Detected {len(tbl)} sources (segmentation-based).')

# Filter out extremely large or extremely small sources
max_area = 5000.0
min_area = 5.0
area_col = np.array(tbl['area'])
mask_area = (area_col >= min_area) & (area_col <= max_area)
tbl = tbl[mask_area]
print(f'Kept {len(tbl)} sources after area filter.')

# Use the naive centroid positions from the segmentation catalog
x_centroids = tbl['xcentroid'].data
y_centroids = tbl['ycentroid'].data

# Plot them
plt.figure(figsize=(8,6))
plt.imshow(data_sub, origin='upper', cmap='gray',
           vmin=np.percentile(data_sub, 0.5), vmax=np.percentile(data_sub, 99.5))
apertures = CircularAperture(list(zip(x_centroids, y_centroids)), r=AP_RAD)
apertures.plot(color='lime', lw=1.0)
plt.title('Detected Sources')
plt.colorbar(label='Bg-Subtracted Counts')
plt.show()
print(f'Final star detections: {len(x_centroids)}')

## 3) Load Star Catalog near Header RA/Dec


In [ ]:
ref_catalog = get_star_catalog(
    ra_center=header_ra,
    dec_center=header_dec,
    radius=search_radius_deg,
    max_mag=max_mag,
    local_fits_path=star_catalog_fits
)
print(f'Catalog star count: {len(ref_catalog)} (from RA={header_ra:.2f}, Dec={header_dec:.2f}, radius={search_radius_deg} deg).')
if len(ref_catalog) == 0:
    print("Warning: No stars found in the local catalog for this region.")

## 4) Project the Catalog onto a Tangent Plane
We force the tangent-plane center to match the header RA/Dec.


In [ ]:
def radec_to_tangent(ra_deg, dec_deg, ra0_deg, dec0_deg):
    """
    Project RA/Dec onto a tangent plane centered at (ra0_deg, dec0_deg).
    Returns x, y in degrees.
    """
    ra_rad = np.radians(ra_deg)
    dec_rad = np.radians(dec_deg)
    ra0_rad = np.radians(ra0_deg)
    dec0_rad = np.radians(dec0_deg)

    cos_c = (np.sin(dec0_rad)*np.sin(dec_rad) +
             np.cos(dec0_rad)*np.cos(dec_rad)*np.cos(ra_rad - ra0_rad))
    x = (np.cos(dec_rad)*np.sin(ra_rad - ra0_rad)) / cos_c
    y = (np.sin(dec_rad)*np.cos(dec0_rad) -
         np.cos(dec_rad)*np.sin(dec0_rad)*np.cos(ra_rad - ra0_rad)) / cos_c
    return np.degrees(x), np.degrees(y)

if len(ref_catalog) == 0:
    raise RuntimeError('No stars in the reference catalog => cannot plate-solve.')

ra_vals = ref_catalog['RA']
dec_vals = ref_catalog['Dec']

# Force tangent-plane center to match FITS header RA/Dec
ra_ref_center = header_ra
dec_ref_center = header_dec

print(f'Tangent-plane center => RA={ra_ref_center:.3f}, Dec={dec_ref_center:.3f}')

x_ref_tan, y_ref_tan = radec_to_tangent(ra_vals, dec_vals, ra_ref_center, dec_ref_center)
ref_points = np.column_stack([x_ref_tan, y_ref_tan])
img_points = np.column_stack([x_centroids, y_centroids])

print(f'Catalog points shape: {ref_points.shape}, Image points shape: {img_points.shape}')

## 5) Astroalign to Match Star Centroids


In [ ]:
try:
    transform, (matched_src, matched_dst) = astroalign.find_transform(img_points, ref_points)
    print("Astroalign transform found.")
    print(f"  Rotation (radians): {transform.rotation:.5f}")
    print(f"  Scale: {transform.scale:.5f}")
    print(f"  Translation (dx,dy): {transform.translation}")
    print(f"  Number of matched stars: {len(matched_src)}")
except Exception as e:
    transform = None
    print("Astroalign failed:", e)


### Plot the matched star centroids in the image

In [ ]:
if transform is not None and len(matched_src) > 0:
    plt.figure(figsize=(8,6))
    plt.imshow(data_sub, origin='upper', cmap='gray',
               vmin=np.percentile(data_sub, 0.5), vmax=np.percentile(data_sub, 99.5))
    plt.colorbar(label='Counts (bg-subtracted)')
    matched_apertures = CircularAperture(matched_src, r=AP_RAD)
    matched_apertures.plot(color='red', lw=1.0)
    plt.title('Stars Used by Astroalign (Red)')
    plt.show()

## 6) Build a TAN WCS from the Transform

In [ ]:
def build_wcs_from_transform(transform, ra_center, dec_center, image_width, image_height):
    w = WCS(naxis=2)
    w.wcs.ctype = ["RA---TAN", "DEC--TAN"]
    w.wcs.crpix = [image_width/2.0, image_height/2.0]
    w.wcs.crval = [ra_center, dec_center]

    cdelt = transform.scale  # deg/pixel in tangent plane
    theta = -transform.rotation
    cos_t = np.cos(theta)
    sin_t = np.sin(theta)
    cd11 = cdelt * cos_t
    cd12 = -cdelt * sin_t
    cd21 = cdelt * sin_t
    cd22 = cdelt * cos_t
    w.wcs.cd = np.array([[cd11, cd12], [cd21, cd22]])
    return w

if transform is not None:
    image_h, image_w = data.shape
    new_wcs = build_wcs_from_transform(
        transform,
        ra_ref_center,
        dec_ref_center,
        image_w,
        image_h
    )
    print("\nBuilt new TAN WCS based on the matched transform.")
else:
    new_wcs = None
    print("No WCS built (Astroalign transform not found).")

## 7) Debug & Verify the Plate Solution
We compute:
- **Tangent-plane RMS**: The distance between matched image stars (transformed) and matched reference stars, in degrees.
- **On-sky RMS**: Convert matched image stars to RA/Dec via new WCS, compare with the reference star’s RA/Dec, measure the difference in arcseconds.


In [ ]:
if new_wcs is not None and transform is not None and len(matched_src) > 0:
    ####################################################
    # 7a) TANGENT-PLANE RMS
    ####################################################
    matched_src_arr = np.array(matched_src)
    matched_dst_arr = np.array(matched_dst)
    predicted_dst = transform(matched_src_arr)
    residuals_deg = matched_dst_arr - predicted_dst
    dist_sq = np.sum(residuals_deg**2, axis=1)
    rms_tan_deg = np.sqrt(np.mean(dist_sq))
    print(f"\nTangent-plane RMS: {rms_tan_deg:.6f} deg = {rms_tan_deg*3600:.3f} arcsec")

    ####################################################
    # 7b) ON-SKY RMS
    ####################################################
    def nearest_index_2d(array2d, point):
        deltas = array2d - point
        dist2 = np.sum(deltas**2, axis=1)
        return np.argmin(dist2)

    ref_points_arr = np.column_stack([x_ref_tan, y_ref_tan])
    ra_all = np.array(ra_vals)
    dec_all = np.array(dec_vals)

    on_sky_diffs_arcsec = []
    for (x_img, y_img), (x_dst, y_dst) in zip(matched_src_arr, matched_dst_arr):
        idx = nearest_index_2d(ref_points_arr, [x_dst, y_dst])
        true_ra = ra_all[idx]
        true_dec = dec_all[idx]
        plate_ra, plate_dec = new_wcs.all_pix2world(x_img, y_img, 0)

        skycoord_true = SkyCoord(true_ra*u.deg, true_dec*u.deg)
        skycoord_plate = SkyCoord(plate_ra*u.deg, plate_dec*u.deg)
        sep_arcsec = skycoord_true.separation(skycoord_plate).arcsecond
        on_sky_diffs_arcsec.append(sep_arcsec)

    if len(on_sky_diffs_arcsec) > 0:
        sky_rms = np.sqrt(np.mean(np.array(on_sky_diffs_arcsec)**2))
        print(f"On-sky RMS: {sky_rms:.3f} arcsec over {len(on_sky_diffs_arcsec)} matched stars")
    else:
        print("No matched stars for on-sky RMS calculation.")
else:
    print("No transform or no matched stars => cannot compute RMS.")

## 8) Compute Alt/Az from RA/Dec, Time, and Location
We show how to interpret the telescope pointing in local coordinates.

In [ ]:
from datetime import timezone

if (header_ra != 0.0) or (header_dec != 0.0):
    planets = load('de421.bsp')
    earth = planets['earth']
    site = earth + wgs84.latlon(obs_lat_deg, obs_lon_deg, elevation_m=obs_elev_m)
    ts = load.timescale()
    
    t_obs = ts.from_datetime(obs_time.replace(tzinfo=timezone.utc))

    star_obj = Star(ra_hours=(header_ra / 15.0), dec_degrees=header_dec)
    app = site.at(t_obs).observe(star_obj).apparent()
    alt, az, distance = app.altaz()

    print(f"\nLocal Alt/Az for RA={header_ra:.3f}°, Dec={header_dec:.3f}° at {obs_time}:")
    print(f"  Alt = {alt.degrees:.2f}°, Az = {az.degrees:.2f}°")
else:
    print("Skipping Alt/Az since RA/Dec=0.0 in FITS header.")

## 9) Optional Debug Plot: Overlay Plate-Solved Starfield
If the plate solve was successful (we have a valid WCS), we can take **all** the star catalog RA/Dec, transform them into pixel coords, and plot them. This helps visualize alignment.

In [ ]:
if new_wcs is not None:
    # Convert entire reference catalog to pixel coords.
    all_cat_ra = np.array(ref_catalog['RA'])
    all_cat_dec = np.array(ref_catalog['Dec'])
    x_pred, y_pred = new_wcs.all_world2pix(all_cat_ra, all_cat_dec, 0)

    plt.figure(figsize=(8,6))
    plt.imshow(data_sub, origin='upper', cmap='gray',
               vmin=np.percentile(data_sub, 0.5), vmax=np.percentile(data_sub, 99.5))
    plt.colorbar(label='Counts (bg-subtracted)')

    # Plot the entire starfield from the plate-solved reference
    cat_apertures = CircularAperture(list(zip(x_pred, y_pred)), r=10)
    cat_apertures.plot(color='blue', lw=1.0)

    plt.title('Plate-Solved Starfield Overlay (Blue)')
    plt.show()
else:
    print("No WCS => cannot overlay the plate-solved starfield.")

### Conclusion
- We now have a final debug plot overlaying **all** catalog stars in blue, as predicted by the newly solved WCS.
- You can compare these with the actual star detections (e.g. in green or red) to verify correct alignment.
